# NoteBook To Check Point or Polygon Inside Polygon

The objective of this notebook is to search a set of points or polygons for spatial overlap with a largerset

By Cascsde Tuholske 2019-01-17

#### Notes
1. Points should be dialated to circles to prevent one falling just outside the larger polygon set (see Lusaka)

### Depedencies

In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry import mapping
from shapely.geometry import Polygon
from shapely.geometry import shape
import ast
from shapely.geometry import mapping
import rasterio
import copy

### Functions

In [2]:
def load_points (file):
    """ This function loads a csv 
    of points and turns it into shapely points"""
    df = pd.read_csv(file)

    # creating a geometry column 
    geometry = [Point(xy) for xy in zip(df['lon'], df['lat'])]

    # Coordinate reference system : WGS84
    crs = {'init': 'epsg:4326'}

    # Creating a Geographic data frame 
    point_gdf = gpd.GeoDataFrame(df, crs=crs, geometry=geometry)
    
    return point_gdf

In [3]:
def point_buffer(gpd_df, raduis):
    "Function to make a shapely polygon buffer around a point"
   
    # AGU 2018-12-04 - radius set to ~250m at the equator 
    #radius = 250*1/(111*1000)

    # new_gpd_df = gpd.GeoDataFrame()
    gpd_df.rename(columns={'geometry':'old_geom'}, inplace=True)
    
    arr = []
    
    for point in gpd_df['old_geom']:
        buffer = point.buffer(radius)
        arr.append((buffer))
    
    gpd_df['geometry'] = arr
    
    return gpd_df

In [4]:
def poly_point (poly, point):
    """
    This function will check if points are inside polygons if given two gpd dataframes with points and polygons
    Returns point ids, point geometry, polygon index # and polygon geometry
    """
    
    out_arr = [] #return an array <<< ---------------- ASK RYAN IF BETTER DO USE DICT 
    
    for index_point, row_point in point.iterrows():
        for index_poly, row_poly in poly.iterrows():
            if row_point['geometry'].within(row_poly['geometry']):
                point_id = row_point['id']
                point_geom = mapping(row_point['geometry']) # makes a dict w/ keys : type and cood
                poly_id = index_poly
                poly_geom = mapping(row_poly['geometry']) # makes a dict w/ keys : type and cood
                
                out_arr.append((point_id, 
                                point_geom, 
                                poly_id, 
                                poly_geom))

    return out_arr

# Note 2018-11-30 update arr to gpd_df 

In [5]:
def poly_overlap (point_buffer, poly_raster):
    """
    This function will check if point buffers intersect with polygons 
    if given two gpd dataframes with point buffers and polygons
    Returns point ids, point geometry, polygon index #, and polygon geometry in a geopandas DF.
    It goes faster if smaller list goes first
    
    Change TOWN & CITY ARRA
    """
    
    # make arrays to fill
    country_arr = []
    city_arr = []
    osm_type_arr = []
    lat_arr = []
    lon_arr = []
    osm_id_arr = [] 
    FID_arr = [] 
    poly_geom_arr = []
    #point_geom_arr = []
    
    for index_point_buffer, row_point_buffer in point_buffer.iterrows():
        for index_poly_raster, row_poly_raster in poly_raster.iterrows():
            if row_point_buffer['geometry'].intersects(row_poly_raster['geometry']):
                country = row_point_buffer['country']
                city = row_point_buffer['town'] # Need to change town vs city
                osm_type = row_point_buffer['osm_type']
                lat = row_point_buffer['lat']
                lon = row_point_buffer['lon']
                osm_id = row_point_buffer['osm_id']
                poly_id = row_poly_raster['FID']
                poly_geom = shape(mapping(row_poly_raster['geometry'])) # make polygon
                #point_geom = row_point_buffer['old_geom']

                country_arr.append((country))
                city_arr.append((city))
                osm_type_arr.append((osm_type))
                lat_arr.append((lat))
                lon_arr.append((lon))
                osm_id_arr.append((osm_id))
                FID_arr.append((poly_id))
                poly_geom_arr.append((poly_geom))
                #point_geom_arr.append((point_geom))
    
    # put results into a geopandas df
    new_gpd_df = gpd.GeoDataFrame()
    new_gpd_df['osm_id'] = osm_id_arr
    new_gpd_df['FID'] = FID_arr
    new_gpd_df['geometry'] = poly_geom_arr
    new_gpd_df['country'] = country_arr           
    new_gpd_df['city'] = city_arr
    new_gpd_df['osm_type'] = osm_type_arr
    new_gpd_df['lat'] = lat_arr
    new_gpd_df['lon'] = lon_arr
    #new_gpd_df['point_geom'] = point_geom_arr
    
    
    return new_gpd_df

# Note 2018-11-30 update arr to gpd_df 

### Load Data

In [13]:
import os
os.getcwd()

'/tana-crunch/cascade/projects/Pop-ERL/notebooks/jupyter/ERL19'

In [18]:
# File Paths
temp_data = '../../../temp_data/ERL19/'
erl_v2_data  = '../../../temp_data/ERL19v2/'

In [24]:
# Load Polygons

# Load Points
points = gpd.read_file(erl_v2_data+'20190430_osm_All.shp')
print(len(points))

# Load Polygons
polygons = gpd.read_file(temp_data+'AFR_PPP_2015_adj_v2_1500c300.shp')
print(len(polygons))

#Out File Name
out = 'WP2015_1500c300_polypoints'

9813
12429


### Buffer Points

In [23]:
# AGU 2018-12-04 - radius set to ~250m at the equator 

radius = 250*1/(111*1000)
radius

0.0022522522522522522

In [26]:
osm_buffer_gdf = point_buffer(points, radius)

In [27]:
osm_buffer_gdf.head(6)

,country,osm_id,town,lat,lon,osm_type,city,old_geom,geometry
0,Algeria,89369215,Tamanrasset,22.785454,5.532446,town,None,POINT (5.5324465 22.7854543),"POLYGON ((5.534698752252252 22.7854543, 5.5346..."
1,Algeria,89980948,In Salah,27.195033,2.482613,town,None,POINT (2.4826132 27.1950331),"POLYGON ((2.484865452252253 27.1950331, 2.4848..."
2,Algeria,252600742,Boumerdès,36.758882,3.470596,town,None,POINT (3.470596 36.758882),"POLYGON ((3.472848252252252 36.758882, 3.47283..."
3,Algeria,253167052,Thenia,36.724986,3.556935,town,None,POINT (3.556935 36.724986),"POLYGON ((3.559187252252252 36.724986, 3.55917..."
4,Algeria,253167208,Zemmouri,36.786406,3.601221,town,None,POINT (3.6012209 36.7864064),"POLYGON ((3.603473152252253 36.7864064, 3.6034..."
5,Algeria,253291208,Lakhdaria,36.563944,3.596907,town,None,POINT (3.596907 36.5639442),"POLYGON ((3.599159252252252 36.5639442, 3.5991..."


In [28]:
osm_buffer_gdf = osm_buffer_gdf.drop(['old_geom'], axis=1)
osm_buffer_gdf.head(6)

,country,osm_id,town,lat,lon,osm_type,city,geometry
0,Algeria,89369215,Tamanrasset,22.785454,5.532446,town,None,"POLYGON ((5.534698752252252 22.7854543, 5.5346..."
1,Algeria,89980948,In Salah,27.195033,2.482613,town,None,"POLYGON ((2.484865452252253 27.1950331, 2.4848..."
2,Algeria,252600742,Boumerdès,36.758882,3.470596,town,None,"POLYGON ((3.472848252252252 36.758882, 3.47283..."
3,Algeria,253167052,Thenia,36.724986,3.556935,town,None,"POLYGON ((3.559187252252252 36.724986, 3.55917..."
4,Algeria,253167208,Zemmouri,36.786406,3.601221,town,None,"POLYGON ((3.603473152252253 36.7864064, 3.6034..."
5,Algeria,253291208,Lakhdaria,36.563944,3.596907,town,None,"POLYGON ((3.599159252252252 36.5639442, 3.5991..."


In [30]:
points[points['osm_type'] == 'city']

,country,osm_id,town,lat,lon,osm_type,city,old_geom,geometry
8863,Algeria,27564946,None,36.364519,6.608260,city,Constantine,POINT (6.60826 36.364519),"POLYGON ((6.610512252252253 36.364519, 6.61050..."
8864,Algeria,27565103,None,35.703275,-0.649298,city,Oran,POINT (-0.6492975999999999 35.7032751),"POLYGON ((-0.6470453477477477 35.7032751, -0.6..."
8865,Algeria,253292160,None,36.713784,4.049392,city,Tizi Ouzou,POINT (4.0493919 36.7137843),"POLYGON ((4.051644152252252 36.7137843, 4.0516..."
8866,Algeria,262963217,None,33.361177,6.860349,city,El Oued,POINT (6.8603492 33.3611766),"POLYGON ((6.862601452252252 33.3611766, 6.8625..."
8867,Algeria,262964638,None,33.109897,6.066102,city,Touggourt,POINT (6.066102 33.1098968),"POLYGON ((6.068354252252252 33.1098968, 6.0683..."
8868,Algeria,264573224,None,33.806352,2.880862,city,Laghouat,POINT (2.8808616 33.8063518),"POLYGON ((2.883113852252253 33.8063518, 2.8831..."
8869,Algeria,274946937,None,35.213312,4.180970,city,Bou Saâda,POINT (4.180970200000001 35.2133123),"POLYGON ((4.183222452252253 35.2133123, 4.1832..."
8870,Algeria,288387133,None,36.733768,3.086179,city,Kouba,POINT (3.0861789 36.7337682),"POLYGON ((3.088431152252253 36.7337682, 3.0884..."
8871,Algeria,288417925,None,36.265344,2.766957,city,Médéa,POINT (2.766957 36.265344),"POLYGON ((2.769209252252252 36.265344, 2.76919..."
8872,Algeria,288418022,None,36.470165,2.828798,city,Blida,POINT (2.8287985 36.4701645),"POLYGON ((2.831050752252252 36.4701645, 2.8310..."


### Intersection

In [ ]:
# Load Polygons

polygons = 'GHS_POP_GPW42000_GLOBE_R2015A_54009_1k_v1_0_Clip_1500c300'

raster_polygons = gpd.read_file(temp_data+polygons+'.shp')
print(len(raster_polygons))

In [ ]:
raster_polygons.head(6)

#### Change CRS if needed

In [ ]:
raster_polygons.crs

In [ ]:
osm_buffer_gdf.crs

In [ ]:
# For GHS CRS

crs = {'ellps': 'WGS84',
 'lon_0': 0,
 'no_defs': True,
 'proj': 'moll',
 'units': 'm',
 'x_0': 0,
 'y_0': 0}

In [ ]:
raster_polygons.crs = crs

In [ ]:
raster_polygons_crs = raster_polygons.to_crs(osm_buffer_gdf.crs)

In [ ]:
# For GHS make FID and reset crs
FID = list(range(len(raster_polygons)))
raster_polygons['DN'] = FID
raster_polygons.columns.values[0] = "FID"

In [ ]:
raster_polygons.head()

### Current Start 
1. Data: 2019-01-18 @ 6:22 pm
2. Raster Poly File: GHS_POP_GPW42000_GLOBE_R2015A_54009_1k_v1_0_Clip_1500c300.shp
3. OSM Town File: 20190114_osm_africa_towns.csv
4. Clip: [25000:35000]
5. outfile - GHS_POP_GPW42000_GLOBE_R2015A_54009_1k_v1_0_Clip_1500c300_subB1.shp

In [ ]:
# run poly_overlap

import time
checkpoint = time.time()

poly_overlap_out = poly_overlap(osm_buffer_gdf, raster_polygons)

print("elapsed time is: {}s".format(time.time()-checkpoint))
print(len(poly_overlap_out))

In [ ]:
print(len(poly_overlap_out))

In [ ]:
poly_overlap_out.tail(20)

In [ ]:
poly_overlap_out.to_file(erl_v2_data+out+'.shp', driver='ESRI Shapefile')